In [1]:
#library imports
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
#function and model imports
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

#from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score

In [3]:
import tensorflow as tf

C:\Users\dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
def df_shuffle(inp):
    inp=shuffle(inp)
    inp=inp.reset_index(drop=True)
    return inp

In [5]:
train_data=pd.read_csv('../Data/train_LZdllcl.csv')
test_data=pd.read_csv('../Data/test_2umaH9m.csv')
print(train_data.shape)
print(test_data.shape)
#tr_data

(54808, 14)
(23490, 13)


In [6]:
train_data=df_shuffle(train_data)

In [7]:
t=train_data.isnull().sum()
t[t>0]

education               2409
previous_year_rating    4124
dtype: int64

In [185]:
def process_categorical(train,test):    #use this after removing target variable
    fulldata=pd.concat([train,test],axis=0)
    fulldata=fulldata.fillna(-1)
    #fulldata=fulldata.drop('region',axis=1)
    fulldata['region']=fulldata['region'].apply(lambda x : int(x[7:]))
    trainend=len(train)
    onecoded=pd.get_dummies(fulldata)
    return (onecoded[:trainend],onecoded[trainend:])

In [186]:
#drop target variable for preprocessing
X_train=train_data.drop('is_promoted',axis=1)
y_data = train_data
#y_data = train_data.loc[train_data['department']=="R&D"]

In [187]:
xtrainfull,xtestproc=process_categorical(X_train,test_data)

In [188]:
ytrainfull=y_data

In [189]:
#sample train data to test on itself
end = 40000 
xtrain = xtrainfull[:end]
ytrain = ytrainfull[:end]

xvalid = xtrainfull[end:54801] 
yvalid = ytrainfull[end:54801]
xvalid=xvalid.reset_index(drop=True)
yvalid=yvalid.reset_index(drop=True)

In [190]:
xtrain.shape

(40000, 27)

In [191]:
xtrain.head()

,employee_id,region,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,department_Analytics,...,department_Technology,education_-1,education_Bachelor's,education_Below Secondary,education_Master's & above,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
0,16171,27,1,35,4.0,8,0,0,58,0,...,0,0,1,0,0,0,1,0,0,1
1,76636,2,1,31,-1.0,1,0,0,56,0,...,0,0,1,0,0,1,0,1,0,0
2,14565,2,1,38,5.0,5,0,0,58,0,...,0,0,1,0,0,1,0,1,0,0
3,44835,6,1,26,3.0,3,0,0,78,0,...,1,0,1,0,0,1,0,0,0,1
4,40563,22,1,25,-1.0,1,1,0,48,0,...,0,0,1,0,0,0,1,0,0,1


In [192]:
X_train_std=scale(xtrain)
X_test_std=scale(xvalid)
X_sub_std=scale(xtestproc)

In [193]:
X_train_std = xtrain.loc[xtrain['department_Sales & Marketing']==1]
X_test_std = xvalid.loc[xvalid['department_Sales & Marketing']==1]
to_remove =['department_Finance','department_Legal','department_Sales & Marketing','department_Analytics','department_Technology','department_Procurement','department_Operations','department_HR','department_R&D']
X_train_std = X_train_std[X_train_std.columns.difference(to_remove)]
X_test_std = X_test_std[X_test_std.columns.difference(to_remove)]

In [194]:
ytrain = ytrain.loc[ytrain['department']=="Sales & Marketing"]
ytrain = ytrain['is_promoted']

In [195]:
ytrain.shape

(12271,)

In [196]:
yvalid = yvalid.loc[yvalid['department']=="Sales & Marketing"]
yvalid = yvalid['is_promoted']

In [197]:
X_train_std.shape

(12271, 18)

In [198]:
Y_bin_train = pd.get_dummies(ytrain)
Y_bin_test = pd.get_dummies(yvalid)

In [199]:
Y_bin_train.shape

(12271, 2)

In [248]:
# Parameters
learning_rate =0.005 #0.02
#num_steps = 500
batch_size = 520#128
#display_step = 100

# Network Parameters
n_hidden_1 = 30#36 # 1st layer number of neurons
n_hidden_2 = 20#24 # 2nd layer number of neurons
n_hidden_3 = 10#12
num_input = 18 # MNIST data input (img shape: 28*28)
num_classes = 2 # MNIST total classes (0-9 digits)


# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [249]:
# Store layers weight & bias
inz = tf.contrib.layers.xavier_initializer()
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [250]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    #layer_2 = tf.nn.dropout(layer_2, 0.9)
    # Output fully connected layer with a neuron for each class
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3) 
    
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

In [251]:
# Construct model
logits = neural_net(X)


class_weights = tf.constant([[1.0, 20.0]])
# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#    logits=logits, labels=Y))
loss_op = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(
    logits=logits, targets=Y,pos_weight=class_weights))


# your class weights
#class_weights = tf.constant([[1.0, 3.0]])
# deduce weights for batch samples based on their true label
#weights_imb = tf.reduce_sum(class_weights * Y, axis=1)
# compute your (unweighted) softmax cross entropy loss
#unweighted_losses = tf.nn.softmax_cross_entropy_with_logits(logits= logits,labels=Y)
# apply the weights, relying on broadcasting of the multiplication
#weighted_losses = unweighted_losses * weights_imb
# reduce the result to get your final loss
#loss_op = tf.reduce_mean(weighted_losses)





optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
pred_tf=tf.argmax(logits, 1)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [252]:
# Start training

no_iterations = int(end/batch_size)-2
print(no_iterations)

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, 30):
        for no_batch in range(0,no_iterations):
            
            batch_x = X_train_std[(no_batch*batch_size) :((no_batch+1)*batch_size) ]
            batch_y = Y_bin_train[(no_batch*batch_size) :((no_batch+1)*batch_size) ]
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            #if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_train_std,Y: Y_bin_train})
        print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))
        print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test_std,
                                      Y: Y_bin_test}))
    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test_std,
                                      Y: Y_bin_test}))
    test_pred= sess.run(pred_tf, feed_dict={X: X_test_std,Y: Y_bin_test})
    tr_pred= sess.run(pred_tf, feed_dict={X: X_train_std,Y: Y_bin_test})
    sub_pred= sess.run(pred_tf, feed_dict={X: X_sub_std})

74
Step 1, Minibatch Loss= 0.9698, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 2, Minibatch Loss= 0.9128, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 3, Minibatch Loss= 0.9138, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 4, Minibatch Loss= 0.9143, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 5, Minibatch Loss= 0.9150, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 6, Minibatch Loss= 0.9153, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 7, Minibatch Loss= 0.9154, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 8, Minibatch Loss= 0.9152, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 9, Minibatch Loss= 0.9149, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 10, Minibatch Loss= 0.9145, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 11, Minibatch Loss= 0.9142, Training Accuracy= 0.930
Testing Accuracy: 0.92336327
Step 12, Minibatch Loss= 0.9139, Training Accurac

ValueError: Cannot feed value of shape (23490, 27) for Tensor 'Placeholder_40:0', which has shape '(?, 18)'

In [247]:
precision, recall, fscore, support = score(ytrain, tr_pred)
accuracy = accuracy_score(ytrain,tr_pred)
print('Accuracy: {}'.format(accuracy))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
#print('support: {}'.format(support))

Accuracy: 0.9295900904571754
precision: [0.92987606 0.42857143]
recall: [0.99964937 0.00347625]
fscore: [0.96350118 0.00689655]


In [28]:
#df_met=pd.DataFrame(all_metrics,columns=['C_name','Accuracy','Notpromoted_rec','Promoted_rec','notpromoted_prec','promoted_prec','fscore_nonprom','fscore_prom'])

NameError: name 'all_metrics' is not defined

In [ ]:
t=df_met.sort_values('Promoted_rec',ascending=True)
t[['C_name','Accuracy','Promoted_rec','promoted_prec','fscore_prom','notpromoted_prec','Notpromoted_rec','fscore_nonprom']]

In [ ]:
#df_met.to_csv('../col_wise_res/col_wise_1_4high_v1.csv',index=False)

In [ ]:
import old

In [ ]:
precision, recall, fscore, support = score(ytrain, tr_pred)
accuracy = accuracy_score(ytrain,tr_pred)
print('Accuracy: {}'.format(accuracy))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
#print('support: {}'.format(support))

In [ ]:
precision, recall, fscore, support = score(yvalid, test_pred)
accuracy = accuracy_score(yvalid,test_pred)
print('Accuracy: {}'.format(accuracy))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
#print('support: {}'.format(support))

In [ ]:
sub_data= {'employee_id':xtestproc['employee_id'],'is_promoted':sub_pred}

In [ ]:
res = pd.DataFrame(sub_data)

In [ ]:
res.to_csv("../results/pred3normal_nn.csv",index=False)

In [ ]:
import stop

In [ ]:
xtrain.isnull().sum().sum()

In [ ]:
#rf = RandomForestClassifier(n_estimators = 20, random_state = 0,max_depth=None,min_samples_split=4)

In [ ]:
#rf = DecisionTreeClassifier(criterion = "gini", max_depth=None,min_samples_split=4,random_state = 0)

In [ ]:
rf = svm.SVC(kernel='linear', C=500, gamma=0.001)

In [ ]:
rf.fit(xtrain,ytrain)

In [ ]:
tr_pred = rf.predict(xtrain)

In [ ]:
valid_pred = rf.predict(xvalid)

In [ ]:
precision, recall, fscore, support = score(ytrain, tr_pred)
accuracy = accuracy_score(ytrain,tr_pred)
print('Accuracy: {}'.format(accuracy))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
#print('support: {}'.format(support))

In [ ]:
precision, recall, fscore, support = score(yvalid, valid_pred)
accuracy = accuracy_score(yvalid,valid_pred)
print('Accuracy: {}'.format(accuracy))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
#print('support: {}'.format(support))

In [ ]:
tr_data['is_promoted'].value_counts()

In [ ]:
tr_data['department'].unique()

In [ ]:
ts_data.shape

In [ ]:
l1=ts_data['employee_id'].values
l2=tr_data['employee_id'].values
common=np.intersect1d(l1,l2)

In [ ]:
common

In [ ]:
np.sort(l1)

In [ ]:
np.sort(l2)